In [1]:
import pandas as pd, numpy as np
import plotly.express as px

In [126]:
# read data
df_historic = pd.read_csv('./data/women_in_parliament-historical_database-1945_to_2018_cleaned.csv')
df_historic = df_historic.query("chamber_type != 'upper' ")
df_regions = df_historic[['country','region']]

df_2021 = pd.read_csv('./data/women_percent_as_of2021_cleaned.csv')
df_2021.rename(columns={'Country':'country'},inplace=True)

df_regions.set_index(['country'], inplace=True)
df_2021.set_index(['country'], inplace=True)

df_2021_regions = df_2021.join(df_regions).reset_index()
df_2021_regions.drop_duplicates(inplace=True)



In [142]:
to_replace = dict({'Azerbaijan':'ASIA',
'Bangladesh':'ASIA',
'Bahrain':'MENA',
'Bahamas': 'AME',
'Bolivia (Plurinational State of)':'AME',
'Central African Republic':'SUB-SAHARAN',
'Chad':'SUB-SAHARAN',
'Chile':'AME',
'China':'ASIA',
'Congo':'SUB-SAHARAN',
"Côte d'Ivoire":'SUB-SAHARAN',
'North Macedonia': 'EUR',
'Venezuela (Bolivarian Republic of)':'AME' })

for i in range(len(df_2021_regions)):
    country  = df_2021_regions['country'].iloc[i]
    if country in to_replace.keys():
        df_2021_regions['region'].iloc[i] = to_replace[country]

/Users/dimitramavridou/Desktop/women-repo/women-in-parliament/myenv/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [149]:
df_2021_regions.to_csv('./data/women_percent_as_of2021_with_regions.csv')

In [148]:

fig = px.treemap(df_2021_regions, 
                 path=[px.Constant("world"), 'region', 'country'], 
                 values='%W',
                 color='%W', 
                 title='Women Representation in Parliament 2021',
                 color_continuous_scale='RdBu',
                 color_continuous_midpoint=np.average(df_2021_regions['%W'], weights=df_2021_regions['%W']),
                 hover_data = ['%W'])
fig.update_traces(root_color="red")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.data[0].hovertemplate = '%{label}<br>%{value}'
fig.show()

In [13]:
fig = px.bar(df_18, x='country', y='women_perc',
            labels=dict(country="Country", women_perc="Representation (%)"),
            title='Women Represetation in Parliament (2018) ')

# Add dropdown
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=["region","ASIA"],
                    label="ASIA",
                    method="update"
                ),
                dict(
                    args=["region", "EUR"],
                    label="EUR",
                    method="update"
                ),
                dict(
                    args=["region", "MENA"],
                    label="MENA",
                    method="update"
                ),
                dict(
                    args=["region", "PAC"],
                    label="PAC",
                    method="update"
                ),
                dict(
                    args=["region", "AME"],
                    label="AME",
                    method="update"
                )
            ]),
            direction="down",
            pad={"r": 2, "t": 2},
            showactive=True,
            x=0.0001,
            xanchor="center",
            y=1.2,
            yanchor="top"
        ),
    ]
)

fig.show()